In [ ]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

In [ ]:
# Required variables and objects
sage_session = sagemaker.session.Session()
s3_output_path = f's3://sagemaker-cmcollander/'

# Training settings
instance_type = "ml.m4.xlarge"
max_jobs = 200
max_parallel_jobs = 10

# IAM role
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()
print("Using IAM role arn: {}".format(role))

In [ ]:
%%time

cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = "sagemaker-roboschool-ray-%s" % cpu_or_gpu
docker_build_args = {
    'CPU_OR_GPU': cpu_or_gpu, 
    'AWS_REGION': boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using ECR image %s" % custom_image_name)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# The hyperparameters we're going to tune
hyperparameter_ranges = {
    'rl.training.config.clip_param': ContinuousParameter(0.1, 0.4),
    'rl.training.config.kl_coeff': ContinuousParameter(0.5, 1.0),
    'rl.training.config.num_sgd_iter': IntegerParameter(3, 50)
}

# The hyperparameters that are the same for all jobs
static_hyperparameters = {
    "rl.training.stop.time_total_s": 900,  # Tell each training job to stop after 10 minutes
    "model_dir": "model",
}

In [ ]:
metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)
estimator = RLEstimator(entry_point="train-reacher.py",
                        source_dir='src',
                        dependencies=["common/sagemaker_rl"],
                        image_name=custom_image_name,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name="reacher-hyper",
                        metric_definitions=metric_definitions,
                        hyperparameters=static_hyperparameters,
                    )

tuner = HyperparameterTuner(estimator,
                            objective_metric_name='episode_reward_mean',
                            objective_type='Maximize',
                            hyperparameter_ranges=hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs,
                            base_tuning_job_name="reacher-hyper-tune",
                           )
tuner.fit()